In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio 
from scipy.fft import fft, fftfreq
from scipy import signal
from scipy.signal import lfilter, filtfilt ,freqz, firwin, freqz
import noisereduce as nr

/Users/panya/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
LimitIndex = 22000
# Load the Drone audio file
audio_file = './sound/Drone.wav'  # Replace with your audio file path
y, fs = librosa.load(audio_file)  # , duration=20)

audio_Drone = y[:LimitIndex]
timesDuration = librosa.get_duration(y=audio_Drone, sr=fs)

print(f'Sampling Rate: {fs} Hz')
print(f'Audio Duration: {timesDuration:.0f} seconds')

Sampling Rate: 22050 Hz
Audio Duration: 1 seconds


In [9]:
Audio(data=audio_Drone, rate=fs)

In [10]:
# Load the NoDrone audio file
audio_file = './sound/NoDrone.wav'  # Replace with your audio file path
y, fs = librosa.load(audio_file)  # , duration=20)
audio_NoDrone = y[:LimitIndex]
timesDuration = librosa.get_duration(y=audio_NoDrone, sr=fs)

print(f'Sampling Rate: {fs} Hz')
print(f'Audio Duration: {timesDuration:.0f} seconds')


Sampling Rate: 22050 Hz
Audio Duration: 1 seconds


In [11]:
Audio(data=audio_NoDrone, rate=fs)

# Spectrogram Function
Calculate Spectrogram by using SFTF method

In [8]:
# Calculate Spectrogram by using SFTF method
def spectrogram_cal(data,fs):
    ms = librosa.feature.melspectrogram(y=data, sr=fs, n_fft=2048, hop_length=128, n_mels=256)
    spectrogram_db = librosa.power_to_db(ms, ref=np.max)

    # n_fft = 1024       # Length of FFT window
    # hop_length = 64   # Number of samples between frames
    # win_length = 1024  # Length of the window
    # window = 'hann'    # Windowing function

    # # Compute the STFT
    # spectrogram = librosa.stft(data, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window)
    # spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram))  # Convert to dB
    
    return spectrogram_db

In [ ]:
# ------- Plot Spectrogram ---------
spectrogram_Drone = spectrogram_cal(audio_Drone)
print("Shape of spectrogram :" + str(spectrogram_Drone.shape))

plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)

librosa.display.specshow(spectrogram_Drone, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of Drone Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

spectrogram_NoDrone = spectrogram_cal(audio_NoDrone)
print("Shape of spectrogram :" + str(spectrogram_NoDrone.shape))

plt.subplot(2, 1, 2)
librosa.display.specshow(spectrogram_NoDrone, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of NoDrone Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# FFT Calcutor Function
1-D discrete Fourier transforms 

In [10]:
# ----- 1-D discrete Fourier transforms ------
def audioFFT_cal (data,fs):
    N = int(fs * timesDuration)         #   Number of sample points

    #yf = fft(audio_DroneFilter)
    #xf = fftfreq(N, 1 / fs)

    T = 1.0 / (fs)   # sample spacing
    x = np.linspace(0.0, N*T, N, endpoint=False)
    yf = fft(data)
    Xf = fftfreq(N, T)[:N//2]
    FFT_Amplitude = 10*np.log(np.abs(yf[0:N//2]))
    #20*np.log10(np.abs(h))
    #max_freq = np.argmax(N * np.abs(yf[0:N//2]))
    #print("High Amplitude Frequency :", max_freq)
    
    return Xf,FFT_Amplitude

In [ ]:
y_signal = audio_Drone

plt.figure(figsize=(10, 8))

# ----- Plot Audio Waveform  -----
plt.subplot(2, 2, 1)
plt.title(f'Audio Waveform')
plt.plot(np.linspace(0, len(y_signal) / fs, len(y_signal)), y_signal)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()
# ----- Plot FFT  -----
xf,yf = audioFFT_cal(y_signal,fs)    
plt.subplot(2, 2, 2)
plt.title(f'FFT waveform')
plt.plot(xf, yf)
plt.grid()
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)

# ------- Plot Spectrogram ---------
spectrogram_db = spectrogram_cal(y_signal)
plt.subplot(2, 1, 2)
plt.title(f'Spectrogram')
librosa.display.specshow(spectrogram_db, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram (STFT configured)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Audio Filter Function
using High Pass and Band Pass Filter

In [12]:
# Audio High Pass Filter using Rectangular window method
# window type: blackman , rectangular, hamming
# fs = Sample rate, Hz
    
def initHPF(cutoff,fs):
    # cutoff = cutoff frequency, Hz
    numtaps = 401  # Size of the FIR filter
    taps = firwin(numtaps, cutoff, window='blackman', pass_zero=False, fs=fs)
    return taps

def initBPF(band,fs):
    #band = [200, 400]  # Desired band-pass range, Hz
    numtaps = 401           # Size of the FIR filter
    taps = firwin(numtaps, band, window='hamming', pass_zero=False, fs=fs)

    return taps

In [ ]:
# Compute and plot the frequency response of the filter

cutoff_freq = 500
HPF_Cof = initHPF(cutoff_freq,fs)
w, h = freqz(HPF_Cof, 2, worN=2000)

# Lowcut = 4000
# Highcut = 10000
# BPF_Cof = initBPF([Lowcut,Highcut],fs)
# w, h = freqz(BPF_Cof, 1, worN=2000)

# Plot the frequency response
plt.figure()
plt.title('Digital filter frequency response')
plt.plot(0.5*fs*w/np.pi, 20*np.log10(np.abs(h)), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.ylim([-150, 10])
plt.xlim([0, 0.5*fs/2])
plt.grid()

plt.show()

In [ ]:
y = audio_Drone
#audio_Filter = filtfilt(HPF_Cof, 1, y)
audio_DroneFilter = lfilter(HPF_Cof, 1, y)

plt.figure(figsize=(10, 8))

# ----- Plot Audio Waveform  -----
xf,yf = audioFFT_cal(y,fs)  
plt.subplot(2, 1, 1)
plt.title(f'Original Audio Waveform')
plt.plot(xf, yf)
plt.grid()
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)
plt.xlim(0,fs/2)

# ----- Plot FFT  -----
xf_filter,yf_filter = audioFFT_cal(audio_DroneFilter,fs)    
plt.subplot(2, 1, 2)
plt.title(f'Filter Audio Waveform')
plt.plot(xf_filter, yf_filter)
plt.grid()
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)
plt.xlim(0,fs/2)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
y = audio_NoDrone
#audio_Filter = filtfilt(HPF_Cof, 1, y)
audio_NoDroneFilter = lfilter(HPF_Cof, 1, y)

plt.figure(figsize=(10, 8))

# ----- Plot Audio Waveform  -----
xf,yf = audioFFT_cal(y,fs)  
plt.subplot(2, 1, 1)
plt.title(f'Original Audio Waveform')
plt.plot(xf, yf)
plt.grid()
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)
plt.xlim(0,fs/2)

# ----- Plot FFT  -----
xf_filter,yf_filter = audioFFT_cal(audio_NoDroneFilter,fs)    
plt.subplot(2, 1, 2)
plt.title(f'Filter Audio Waveform')
plt.plot(xf_filter, yf_filter)
plt.grid()
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)
plt.xlim(0,fs/2)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
y = audio_DroneFilter

plt.figure(figsize=(10, 8))

# ----- Plot Audio Waveform  -----
plt.subplot(2, 2, 1)
plt.title(f'Audio Waveform')
plt.plot(np.linspace(0, len(y) / fs, len(y)), y)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()

# ----- Plot FFT  -----
xf,yf = audioFFT_cal(y,fs)    
plt.subplot(2, 2, 2)
plt.title(f'FFT waveform')
plt.plot(xf, yf)
plt.xlabel('Freq (Hz)')
plt.ylabel('Amplitude (dB)')
plt.ylim(-50,80)
plt.grid()

# ------- Plot Spectrogram ---------
spectrogram_db = spectrogram_cal(y)
plt.subplot(2, 1, 2)
plt.title(f'Spectrogram')
librosa.display.specshow(spectrogram_db, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram (STFT configured)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# ------- Plot Spectrogram ---------
spectrogram_Drone = spectrogram_cal(audio_DroneFilter)
print("Shape of spectrogram :" + str(spectrogram_Drone.shape))

plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)

librosa.display.specshow(spectrogram_Drone, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of Drone Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

spectrogram_NoDrone = spectrogram_cal(audio_NoDroneFilter)
print("Shape of spectrogram :" + str(spectrogram_NoDrone.shape))

plt.subplot(2, 1, 2)
librosa.display.specshow(spectrogram_NoDrone, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of NoDrone Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Plot Spectrogram with plt.imshow function

In [ ]:
z_min, z_max = -np.abs(spectrogram_Drone).max(), np.abs(spectrogram_Drone).max() 
c = plt.imshow(spectrogram_Drone, cmap ='viridis', vmin = z_min, vmax = z_max, 
                    interpolation ='nearest', origin ='lower') 
plt.colorbar(c) 
plt.show() 

print(spectrogram_Drone.shape)

In [ ]:
#reduced_noise = nr.reduce_noise(y=audio_Drone, sr=fs, y_noise=audio_NoDrone, prop_decrease=0.7)
#noisereduction
reduced_noise = nr.reduce_noise(y=audio_Drone, sr=fs, stationary=True, prop_decrease=0.95)

# ------- Plot Spectrogram ---------
spectrogram_Drone = spectrogram_cal(audio_Drone)
print("Shape of spectrogram :" + str(spectrogram_Drone.shape))

plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)

librosa.display.specshow(spectrogram_Drone, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of Original Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

spectrogram_reduce_noise = spectrogram_cal(reduced_noise)
print("Shape of spectrogram :" + str(spectrogram_NoDrone.shape))

plt.subplot(2, 1, 2)
librosa.display.specshow(spectrogram_reduce_noise, sr=fs, x_axis='time', y_axis='linear', cmap='viridis')
#cmap = 'viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.colorbar(format='%+2.0f dB')
plt.title(f'Spectrogram of Noise Reduce Sound')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.grid()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
Audio(data=audio_NoDrone, rate=fs)

In [ ]:
Audio(data=reduced_noise, rate=fs)

# Power Spectrum Density (PSD)
compare PSD between sound of Drone and NoDrone

In [ ]:
# Set the parameters for the PSD
NFFT = 2048
noverlap = NFFT // 2

# Compute the PSD of the audio signal
f_Drone, psd_Drone = signal.welch(audio_Drone, fs=fs, nperseg=NFFT, noverlap=noverlap)
f_NoDrone, psd_NoDrone = signal.welch(audio_NoDrone, fs=fs, nperseg=NFFT, noverlap=noverlap)

# Plot the PSD
plt.figure(figsize=(10, 6))
plt.plot(f_Drone, 10*np.log10(psd_Drone),label='Drone')
plt.plot(f_NoDrone, 10*np.log10(psd_NoDrone),label='No Drone')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.legend()
plt.show()

# Identify the noise frequencies
# threshold = np.mean(psd_NoDrone) + np.std(psd_NoDrone)
# noise_freqs = f[psd > threshold]
#print('Noise frequencies:', noise_freqs)